# Get the hex centroids by clicking the center of each hex on the video

We click each hex individually instead of fitting a hex grid because the camera is slightly angled such that the hex grid doesn't fit perfectly onto the maze video. This gives us better results (fitting the hex grid ends up warping space). It's not ideal but we only have to do it once, so it's fine for now!

### Important: hexes MUST be clicked from top top bottom, left to right!
If hex 1 is on top in the maze, this looks like: hex 1, 4, 6, 5, 8, 7, 11, etc.

Click only open hexes (not locations where permanent barriers will be placed).

To undo a click, press `u` for `undo`.

When all hexes have been clicked, press `q` or `escape` to close the video window.

In [ ]:
import cv2
import pandas as pd
import numpy as np
import math

In [ ]:
# Path to the hex maze video to get centroids from
video_file_path = 'video_files/maze_empty.h264'

# Optional, if we don't have one then set it to None
CM_PER_PIXEL = 0.16 

# If we want to save the file
save = False

# Declare frame and clicked_points as global variables
frame = None
clicked_points = []

def get_pixel_coordinates(event, x, y, flags, param):
    """ Callback function to get the (x, y) coordinates of a click """
    if event == cv2.EVENT_LBUTTONDOWN:  # Left mouse button click
        # Print and store the clicked point
        print(f'Clicked at ({x}, {y})')
        clicked_points.append((x, y))
        # Draw a red dot at the clicked position so we know where we have clicked
        cv2.circle(frame, (x, y), radius=5, color=(0, 0, 255), thickness=-1)
        cv2.imshow('Hex maze video', frame)

def undo_last_click():
    """ Undo the last click by removing the coordinates from our list and the plot """
    global frame, clicked_points
    if clicked_points:  # Check if there are any clicked points to undo
        click_to_undo = clicked_points.pop()  # Remove the last clicked point
        print(f"Removed click at ({click_to_undo[0]}, {click_to_undo[1]})")
        # Redraw the frame (remove the dot for the click we undid)
        frame[:] = original_frame.copy()
        for point in clicked_points:  # Redraw all remaining dots
            cv2.circle(frame, point, radius=5, color=(0, 0, 255), thickness=-1)
        cv2.imshow('Hex maze video', frame)

# Load the video file
cap = cv2.VideoCapture(video_file_path)

# Complain if we can't find or open the video file
if not cap.isOpened():
    print(f"Error: Could not open video at {video_file_path}!")
    exit()

# We only need a single frame from the video to get hex positions
frame_number = 100
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)  # Set the frame position
ret, frame = cap.read()

# Complain if we can't read this frame
if not ret:
    print(f"Error: Could not read frame {frame_number}!")
    cap.release()
    exit()

# Set up the named window and attach the mouse callback function
cv2.namedWindow('Hex maze video')
cv2.setMouseCallback('Hex maze video', get_pixel_coordinates)

# Display the selected frame
cv2.imshow('Hex maze video', frame)
original_frame = frame.copy() # Save a copy to use for redrawing if we undo clicks

# Run until the user closes the window
while True:
    key = cv2.waitKey(1) & 0xFF
    # Press 'q' or 'escape' to close the window
    if key == ord('q') or key == 27:
        break
     # Press 'u' to undo the last click
    elif key == ord('u'): 
        undo_last_click()

# Close all windows 
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)  # Add a short delay so the window actually closes

### Save clicked output into an excel file

Assuming all hexes have been correctly clicked IN ORDER from top to bottom, left to right, choose the correct hex list corresponding to the video angle. (If hex 2 is on top in the video, use `HEX_LIST_2`, etc.)

In [ ]:
# Hexes from top to bottom, left to right, if hex 1 is on top
HEX_LIST_1 = [1, 4, 6, 5, 8, 7, 11, 10, 9, 14, 13, 12, 18, 17, 16, 15,
            22, 21, 20, 19, 27, 26, 25, 24, 23, 32, 31, 30, 29, 28,
            38, 37, 36, 35, 34, 33, 49, 42, 41, 40, 39, 48, 2, 47, 46, 45, 44, 43, 3]
# Hexes from top to bottom, left to right, if hex 2 is on top
HEX_LIST_2 = [2, 49, 47, 38, 42, 32, 46, 37, 27, 41, 31, 22, 45, 36, 26, 18, 
              40, 30, 21, 14, 44, 35, 25, 17, 11, 39, 29, 20, 13, 8, 
              43, 34, 24, 16, 10, 6, 48, 28, 19, 12, 7, 4, 3, 33, 23, 15, 9, 5, 1]
# Hexes from top to bottom, left to right, if hex 3 is on top
HEX_LIST_3 = [3, 48, 33, 43, 28, 39, 23, 34, 44, 19, 29, 40, 15, 24, 35, 45, 
              12, 20, 30, 41, 9, 16, 25, 36, 46, 7, 13, 21, 31, 42, 
              5, 10, 17, 26, 37, 47, 4, 8, 14, 22, 32, 49, 1, 6, 11, 18, 27, 38, 2]

# Xulu's video angle shows the maze with hex 2 on the top, so use this list
hex_list = HEX_LIST_2

# Check that we have clicked to indicate exactly one centroid per hex
if len(clicked_points) != len(hex_list):
    print(f"Expected centroids for {len(hex_list)} hexes, but {len(clicked_points)} hexes were clicked!")
    print(f"Please go back and click centroids for exactly {len(hex_list)} hexes.")

# Create a dataframe to save the x, y coordinates for each hex
x_coords, y_coords = zip(*clicked_points)
hex_coordinates_df = pd.DataFrame({'hex': hex_list, 'x': x_coords, 'y': y_coords})

# Optionally, convert the x, y coords in pixels to meters and add that to the dataframe
if CM_PER_PIXEL is not None:
    meters_per_pixel = CM_PER_PIXEL / 100
    hex_coordinates_df['x_meters'] = np.array(x_coords)*CM_PER_PIXEL
    hex_coordinates_df['y_meters'] = np.array(y_coords)*CM_PER_PIXEL
display(hex_coordinates_df)

# Save the dataframe to an excel file and csv
if save:
    save_file_name = 'hex_coordinates'
    hex_coordinates_df.to_excel(f"{save_file_name}.xlsx", index=False)
    hex_coordinates_df.to_csv(f"{save_file_name}.csv", index=False)
    print(f"Hex coordinates saved to {save_file_name}")